In [88]:
import requests
from bs4 import BeautifulSoup

In [89]:
BASE_URL = "https://www.bls.gov"
URL = f"{BASE_URL}/ooh/field-of-degree/home.htm"
field_of_degree_page = requests.get(URL)


In [90]:
soup = BeautifulSoup(field_of_degree_page.content, "html.parser")

In [91]:
field_of_degree_links = []
field_of_degree_list = []
field_of_degree_list_elements = soup.find("table").find("ul").find_all("li")

for field_of_degree_list_element in field_of_degree_list_elements:
    anchor_tag = field_of_degree_list_element.find("a")
    href = anchor_tag.attrs["href"]
    field_of_degree_list.append(anchor_tag.text)
    field_of_degree_links.append(href)

In [92]:
degree_soups = []

for link in field_of_degree_links:
    href_in_question = link
    qualified_href = f'{BASE_URL}{href_in_question}'

    field_of_degree_handbook_page = requests.get(qualified_href)
    degree_soup = BeautifulSoup(field_of_degree_handbook_page.content, "html.parser")

    degree_soups.append(degree_soup)


In [93]:
degree_map = {}

for degree_name in field_of_degree_list:
    degree_map[degree_name] = {}
    degree_map[degree_name]["table_1"] = None
    degree_map[degree_name]["table_2"] = None
    degree_map[degree_name]["chart_1"] = None
    degree_map[degree_name]["chart_2"] = None


In [94]:
table_to_key = {
    "Table 1": "table_1",
    "Chart 1": "table_2",
    "Table 2": "chart_1",
    "Chart 2": "chart_2"
}

for i in range(len(field_of_degree_list)):
    degree_soup = degree_soups[i]
    degree_name = field_of_degree_list[i]

    tables = degree_soup.find_all("table")
    table_dict = {}

    for table in tables:
        table_title = table.find("caption").find("span").text

        if (table_title == "test table"):
            continue

        table_key = table_to_key[table_title.split(".")[0]]

        degree_map[degree_name][table_key] = table
    
    print()

In [95]:
import itertools
import functools
import pandas as pd

In [96]:
# Parse all the table_1
def map_to_table_1(table_dict):
    return table_dict["table_1"]
    
table_1s = list(map(map_to_table_1, degree_map.values()))

all_frames = []

for table in table_1s:
    table_columns = list(map(lambda x: x.text, table.find("thead").find("tr").find_all("th")))
    table_rows = list(map(lambda row: [row.find("th").text.strip("\n")] + list(map(lambda rowelem: rowelem.text, row.find_all("td"))) , table.find("tbody").find_all("tr")))
    table_frame = pd.DataFrame(data=table_rows, columns=table_columns).T
    
    complete_frame = table_frame.drop(index="All fields")[1:]
    complete_frame.reset_index(inplace=True)
    complete_frame.columns = ["Field Of Study", "Employment", "Median wage", "Percent employed part time", "Percent employed in occupations requiring at least a bachelor's degree", "Percent with an advanced degree"]
    all_frames.append(complete_frame)
   
table_1_frame = pd.concat(all_frames)

In [97]:
# Parse all the chart_1
def map_to_chart_1(table_dict):
    return table_dict["chart_1"]
    
chart_1s = list(map(map_to_chart_1, degree_map.values()))

all_frames = []

#for table in chart_1s:
for i in range(len(chart_1s)):
    table = chart_1s[i]
    field_of_study = field_of_degree_list[i]

    table_columns = list(map(lambda x: x.text, table.find("thead").find("tr").find_all("th")))
    table_rows = list(map(lambda row: [row.find("th").text.strip("\n")] + list(map(lambda rowelem: rowelem.text, row.find_all("td"))) , table.find("tbody").find_all("tr")))
    table_frame = pd.DataFrame(data=table_rows, columns=table_columns)
    table_frame["Field of Study"] = field_of_study
 
    all_frames.append(table_frame)


table_2_frame = pd.concat(all_frames)

In [98]:
# Parse all the table_2
def map_to_table_2(table_dict):
    return table_dict["table_2"]
    
table_2s = list(map(map_to_table_2, degree_map.values()))

all_frames = []

#for table in chart_1s:
for i in range(len(table_2s)):

    table = table_2s[i]
    field_of_study = field_of_degree_list[i]

    table_columns = list(map(lambda x: x.text, table.find("thead").find("tr").find_all("th")))
    table_rows = list(map(lambda row: [row.find("th").text.strip("\n")] + list(map(lambda rowelem: rowelem.text.strip("%"), row.find_all("td"))) , table.find("tbody").find_all("tr")))
    table_frame = pd.DataFrame(data=table_rows, columns=table_columns)
    table_frame["Field of Study"] = field_of_study

    table_frame.rename(columns={ table_frame.columns[0]: "Majors" }, inplace = True)
    all_frames.append(table_frame)


table_3_frame = pd.concat(all_frames)

In [104]:
# Parse all the chart_2
def map_to_chart_2(table_dict):
    return table_dict["chart_2"]
    
chart_2s = list(map(map_to_chart_2, degree_map.values()))

all_frames = []

#for table in chart_1s:
for i in range(len(chart_2s)):
    table = chart_2s[i]
    field_of_study = field_of_degree_list[i]

    table_columns = list(map(lambda x: x.text, table.find("thead").find("tr").find_all("th")))
    table_rows = list(map(lambda row: [row.find("th").text.strip("\n")] + list(map(lambda rowelem: rowelem.text.strip("%"), row.find_all("td"))) , table.find("tbody").find_all("tr")))
    table_frame = pd.DataFrame(data=table_rows, columns=table_columns)
    table_frame["Field of Study"] = field_of_study
    
    all_frames.append(table_frame)

table_4_frame = pd.concat(all_frames)

In [106]:
with pd.ExcelWriter("bls-fields-of-study.xlsx") as writer:
    table_1_frame.to_excel(writer, sheet_name="DegreeComparedToOthers_Table1", index=False)
    table_2_frame.to_excel(writer, sheet_name="TopEmployingOccupations_Table2", index=False)
    table_3_frame.to_excel(writer, sheet_name="TypesOfMajorsForField_Table3", index=False)
    table_4_frame.to_excel(writer, sheet_name="EmploymentForField_Table4", index=False)